In [1]:
##-----------------------------------------##
#               Packages
##-----------------------------------------##
import dash_bootstrap_components as dbc
from datetime import datetime as dt
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import dash_table
import dash_leaflet as dl
from dash.dependencies import Output, Input

from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets

import ee
from datetime import date
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

#import eeconvert # Package to conver Earth Engine collection to dataframe or geodataframe
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


# Earth Engine Python API
ee.Initialize()



In [3]:
##-----------------------------------------##
#               Collection Attributes
##-----------------------------------------##


def makeLandsatSeries():

    lt4 = ee.ImageCollection('LANDSAT/LT04/C01/T1_SR')
    lt5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
    le7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
    lc8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

    lt4 = lt4.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    lt5 = lt5.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    le7 = le7.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    lc8 = lc8.select(['B2','B3','B4','B5','B6','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])

    fullCollection = ee.ImageCollection(lt4.merge(lt5).merge(le7).merge(lc8))
    return fullCollection


##-----------------------------------------##
#               Mapbox Attributes
##-----------------------------------------##

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css',
'https://codepen.io/chriddyp/pen/brPBPO.css']


# Mapbox setup
mapbox_url = "https://api.mapbox.com/styles/v1/mapbox/{id}/tiles/{{z}}/{{x}}/{{y}}{{r}}?access_token={access_token}"
mapbox_token = 'pk.eyJ1IjoibmF0MSIsImEiOiJjazhpeDdwZ3gwM3FyM2RueTZwdDJ0bzB2In0.lqoQJb90lcn0cu-zulXWyw'
mapbox_ids = ["light-v9", "dark-v9", "streets-v9", "outdoors-v9", "satellite-streets-v9"]

# Element mapbox_ids
BASE_LAYER_ID = "base-layer-id"
BASE_LAYER_DROPDOWN_ID = "base-layer-drop-down-id"
MAP_ID = "map-id"
MARKER = "marker-id"
NEW_MARKER= "new-marker-id"
COORDINATE_CLICK_ID = "coordinate-click-id"


##-----------------------------------------##
#               Functions
##-----------------------------------------##



app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

controls = dbc.Card(
    [   
        dbc.FormGroup(
            [
                dbc.Label("Base Layers"),
                dcc.Dropdown(
                    id=BASE_LAYER_DROPDOWN_ID,
                    options=[
                        {"label": i, "value": mapbox_url.format(id=i, access_token=mapbox_token)} for i in mapbox_ids
                    ],
                    value=mapbox_url.format(id="light-v9", access_token=mapbox_token)
                ),
            ]
        ),
        
        dbc.FormGroup(
            [
                dbc.Label("Cloud"),
                dbc.Input(id="cloud", type="number", min = 0, max = 100, step = 10,),
            ]
        ),
        html.Div(id="number-out"),
        
        dbc.FormGroup(
            [
                dbc.Label("Date"),
                dcc.DatePickerRange(
                    id='my-date-picker-range',
                    #month_format='Do MMM, YYYY',
                    display_format="YYYY,MM,DD",
                    min_date_allowed=dt(1995,1,1),
                    max_date_allowed=dt.today(),
                    #initial_visible_month=dt(2017, 8, 5),
                    start_date=dt(2000,1,1),
                    #end_date=dt.today(),
                    start_date_placeholder_text="YYYY,MM,DD",
                    end_date_placeholder_text="YYYY,MM,DD",
                ),
                html.Div(id='output-container-date-picker-range')
            ]
        )
        
    ],
    body=True,
)

#html.P("Coordinate (click on map):"),
#html.Div(id=COORDINATE_CLICK_ID),
#fluid=True,
app.layout = dbc.Container(
    [
        html.H1("Satellite Analysis"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dl.Map(id=MAP_ID,
                               style={'width': '1000px', 'height': '500px'},
                               center=[-13, 48],
                               zoom = 5,
                               children=[
                                   dl.TileLayer(id=BASE_LAYER_ID),
                                   dl.Marker(id=MARKER, position=[56, 9.8], interactive=True,   opacity=0.8),
                                   html.Div(id=NEW_MARKER),
                                   html.Div(id=COORDINATE_CLICK_ID)
                               ])),
                
                dbc.Col(controls),
            
            ],),
    ],
)


# MARKER
@app.callback(Output(NEW_MARKER, 'children'),
              [Input(MAP_ID, 'click_lat_lng')])
def new_marker(x):
    if x is not None:

        global lat
        global lon
        lat, lon = x
        #print("Marker:", lat)

        return dl.Marker(position=[lat, lon])
    else:
        return None

# COORDINATES
@app.callback(Output(COORDINATE_CLICK_ID, 'children'),
              [Input(MAP_ID, 'click_lat_lng')])
def click_coord(e):
    if e is not None:
        #print("Click:", json.dumps(e))
        return json.dumps(e)
    else:
        return "-"

# BASE LAYER
@app.callback(Output(BASE_LAYER_ID, "url"),
              [Input(BASE_LAYER_DROPDOWN_ID, "value")])
def set_baselayer(url):
    return url


# CLOUD
@app.callback(
Output("number-out", "children"),
[Input("cloud", "value")
],) 

def number_render(cloud):

    if lat is not None:

        global latitude
        latitude = lat

        global longitude
        longitude = lon

        global Maxcloud
        Maxcloud = cloud 
        point = {'type':'Point', 'coordinates':[longitude, latitude]}; # This point is on turbid water

        #print("Cloud lat:",lat, "Type:", type(lat))
        #print("Cloud latitude:",latitude, "Type:", type(latitude))
        return ()

# DATE
@app.callback(
dash.dependencies.Output('output-container-date-picker-range', 'children'),
[dash.dependencies.Input('my-date-picker-range', 'start_date'),
 dash.dependencies.Input('my-date-picker-range', 'end_date')
])

def update_output(start_date, end_date):

    # Define point
    point = {'type':'Point', 'coordinates':[longitude, latitude]}; # This point is on turbid water
    #print("Update Output Point", point)

    # Define Full Collection
    fullCollection= makeLandsatSeries()

    # Filter collection by Lat, Lon & Cloud
    filtered = fullCollection.filterBounds(ee.Geometry.Point(longitude,latitude)).filter(ee.Filter.lt('CLOUD_COVER', Maxcloud))

    #Count Size // Total number of images: 165
    count = int(filtered.size().getInfo())
    #print("Update Output Count:", count)

    #Info
    info = filtered.getRegion(point,500).getInfo()


    if start_date is not None:
        start_date = dt.strptime(start_date.split('T')[0], '%Y-%m-%d')
        start_date_string = start_date.strftime('%Y-%m-%d')
        startDate = start_date_string

        end_date = dt.strptime(end_date.split('T')[0], '%Y-%m-%d')
        end_date_string = end_date.strftime('%Y-%m-%d')
        endDate = end_date_string
        
        #print("strptime: ", end_date)
        #print("end string: ", end_date_string)
        #print("endDate: ", endDate)

        # Filter Collection with Lat, Lon & Cloud
        filtered2 = filtered.filterDate(startDate, endDate)

        #Count Size // Total number of images:
        count = int(filtered2.size().getInfo())
        print("Your inputs return", count, "images")

        info = filtered2.getRegion(point,500).getInfo()

        # Datframe
        df = pd.DataFrame(info,columns = ['id', 'longitude', 'latitude', 'time', 'blu', 'grn', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa'])
        #df = df.to_json(date_format='iso', orient='split')

        # Create an ID column with NAN values
        df['Satellite_ID'] = ('NAN')

        # Make sure that all NaN values are `np.nan` not `'NAN'` (strings)
        df = df.replace('NAN', np.nan)
        mask = df['id'].str.contains(r'LT04')
        df.loc[mask, 'Satellite_ID'] = ('L4')

        mask = df['id'].str.contains(r'LT05')
        df.loc[mask, 'Satellite_ID'] = ('L5')

        mask = df['id'].str.contains(r'LT05')
        df.loc[mask, 'Satellite_ID'] = ('L5')

        mask = df['id'].str.contains(r'LE07')
        df.loc[mask, 'Satellite_ID'] = ('L7')

        mask = df['id'].str.contains(r'LC08')
        df.loc[mask, 'Satellite_ID'] = ('L8')

        # Drop First row (a duplicate of column header)
        df = df.drop(df.index[0])

        # The Earth Engine time stamp in milliseconds since the UNIX epoch.
        # Link GEE: https://developers.google.com/earth-engine/glossary

        # Convert UNIX to datetime
        # Use pd.to_datetime() to convert unix epoch time
        # Result is like this: 1994-03-17 06:11:20.254
        df['time_stamp'] = pd.to_datetime(df['time'], unit='ms')

        # Convert to String inorder to apply string split
        df['time_string'] = df['time_stamp'].astype(str)

        # make the new date column using string indexing [0:10] will give us the year,month,date part
        # Result is like this 1994-03-17
        df['Date'] = df['time_string'].str[0:10]

        # Convert back to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Drop unnecessary columns
        #df = df.drop('time_stamp', axis=1)
        df = df.drop('time_string', axis=1)

        # Sort by Date
        df = df.sort_values(by='Date')
        

        # Calculate NDWI
        #ndwi = (green - swir) / (green + swir)
        df['NDWI'] = (df['grn'] - df['swir1']) / (df['grn'] + df['swir1'])
        df['NDWI'] = pd.to_numeric(df['NDWI'])
        
        # Set Index
        df = df.set_index('time_stamp')
        #print("Original DF: ", df)

        # Simplified Table for Dash
        #simple_df = df[['NDWI', 'Date']].copy()
        
    
        # New Dataframe: Set Mean NDWI & Time columns. Time column is from index
        yearly_summary = pd.DataFrame()
        yearly_summary['NDWI'] = df.NDWI.resample('Y').mean()

        yearly_summary['Time'] = yearly_summary.index
        print(yearly_summary.head())


        # Create Line Chart
        fig = px.line(yearly_summary,
                      x = yearly_summary['Time'],
                      y = yearly_summary['NDWI'])

        # Fig Update & Set Title
        fig.update_layout(title_text="Yearly Mean NDWI Values")
        fig.update_traces(overwrite=True)


        #return "start: {}, end: {}".format(start_date_string, end_date_string) 
        return [
            dcc.Graph(
                id=column,
                figure = fig
            )
            # check if column exists - user may have deleted it
            # If `column.deletable=False`, then you don't
            # need to do this check.
            for column in ["NDWI"] if column in yearly_summary
        ]

if __name__ == "__main__":
    app.run_server(debug=False, port=8150)
    #app.run_server(debug=True, port=8888)
    
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8150/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Apr/2020 11:46:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:46:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:46:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:46:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:46:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:46:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:46:45] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/nat/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.en

127.0.0.1 - - [14/Apr/2020 11:46:48] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Apr/2020 11:47:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 11:47:33] "POST /_dash-update-component HTTP/1.1" 200 -


Your inputs return 342 images
                NDWI       Time
time_stamp                     
2000-12-31  0.688477 2000-12-31
2001-12-31  0.685324 2001-12-31
2002-12-31  0.719874 2002-12-31
2003-12-31  0.571947 2003-12-31
2004-12-31  0.701617 2004-12-31


127.0.0.1 - - [14/Apr/2020 11:47:49] "POST /_dash-update-component HTTP/1.1" 200 -
